# Packing circuits

This notebook looks at some examples of packing circuits to reduce their ancilla costs. In particular in these examples this problem is solved via the calculation of minimum vertex covers, which is done by not allowing 'evicted gates'.

First, import the modules we need.

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket_dqc.networks import NISQNetwork
from pytket_dqc.allocators import HypergraphPartitioning, Annealing
from pytket_dqc.circuits import HypergraphCircuit, BipartiteCircuit
from pytket_dqc.utils import ebit_memory_required
from pytket.transform import Transform
from pytket_dqc.utils import check_equivalence, DQCPass

## Trivial examples

### Two CZs

In [ ]:
from pytket.circuit import OpType

circuit = Circuit(2)
circuit.add_gate(OpType.CU1, 1.0, [0, 1])
circuit.add_gate(OpType.CU1, 1.0, [0, 1])
network = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit)

In [ ]:
DQCPass().apply(circuit)
dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)
render_circuit_jupyter(circuit)

In [ ]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [ ]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

In [ ]:
orig_qubits = circuit.qubits
new_qubits = bp_circuit.packed_circuit.qubits
qubit_mapping = {
    orig_qubits[0] : new_qubits[0],
    orig_qubits[1] : new_qubits[1],
}
if check_equivalence(circuit, bp_circuit.packed_circuit, qubit_mapping):
    print("The circuit equality has been verified.")
else:
    raise Exception("Circuit equality failed!")

### Two CZs (opposite controls)

In [ ]:
circuit = Circuit(2)
circuit.add_gate(OpType.CU1, 1.0, [0, 1])
circuit.add_gate(OpType.CU1, 1.0, [1, 0])
network = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit)

In [ ]:
DQCPass().apply(circuit)
dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)
render_circuit_jupyter(circuit)

In [ ]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [ ]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

In [ ]:
orig_qubits = circuit.qubits
new_qubits = bp_circuit.packed_circuit.qubits
qubit_mapping = {
    orig_qubits[0] : new_qubits[0],
    orig_qubits[1] : new_qubits[2],
}
if check_equivalence(circuit, bp_circuit.packed_circuit, qubit_mapping):
    print("The circuit equality has been verified.")
else:
    raise Exception("Circuit equality failed!")

### Two CZs with X in between

In [ ]:
circuit = Circuit(2)
circuit.add_gate(OpType.CU1, 1.0, [0, 1])
circuit.H(0).Rz(1, 0).H(0)
circuit.H(1).Rz(1, 1).H(1)
circuit.add_gate(OpType.CU1, 1.0, [1, 0])
network = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit)

In [ ]:
DQCPass().apply(circuit)
dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)
render_circuit_jupyter(circuit)

In [ ]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [ ]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

In [ ]:
orig_qubits = circuit.qubits
new_qubits = bp_circuit.packed_circuit.qubits
qubit_mapping = {
    orig_qubits[0] : new_qubits[2],
    orig_qubits[1] : new_qubits[0],
}
if check_equivalence(circuit, bp_circuit.packed_circuit, qubit_mapping):
    print("The circuit equality has been verified.")
else:
    raise Exception("Circuit equality failed!")

### Two CZs with non-packable in between

In [ ]:
circuit = Circuit(2)
circuit.add_gate(OpType.CU1, 1.0, [0, 1])
circuit.H(0).Rz(0.5, 0).H(0)
circuit.H(1).Rz(0.5, 1).H(1)
circuit.add_gate(OpType.CU1, 1.0, [1, 0])
network = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit)

In [ ]:
DQCPass().apply(circuit)
dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)
render_circuit_jupyter(circuit)

In [ ]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [ ]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

In [ ]:
orig_qubits = circuit.qubits
new_qubits = bp_circuit.packed_circuit.qubits
qubit_mapping = {
    orig_qubits[0] : new_qubits[3],
    orig_qubits[1] : new_qubits[0],
}
if check_equivalence(circuit, bp_circuit.packed_circuit, qubit_mapping):
    print("The circuit equality has been verified.")
else:
    raise Exception("Circuit equality failed!")

### Simple CZ cycle

In [ ]:
circuit = Circuit(3)
circuit.add_gate(OpType.CU1, 1.0, [0, 1])
circuit.add_gate(OpType.CU1, 1.0, [1, 2])
circuit.add_gate(OpType.CU1, 1.0, [2, 0])
network = NISQNetwork(
    [[0,1], [1, 2]],
    {0: [0], 1: [1], 2:[2]}
)
render_circuit_jupyter(circuit)

In [ ]:
DQCPass().apply(circuit)
dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)
render_circuit_jupyter(circuit)

In [ ]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [ ]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

In [ ]:
orig_qubits = circuit.qubits
new_qubits = bp_circuit.packed_circuit.qubits
qubit_mapping = {
    orig_qubits[0] : new_qubits[0],
    orig_qubits[1] : new_qubits[5],
    orig_qubits[2] : new_qubits[3],
}
if check_equivalence(circuit, bp_circuit.packed_circuit, qubit_mapping):
    print("The circuit equality has been verified.")
else:
    raise Exception("Circuit equality failed!")

### Complicated circuit

In [ ]:
circuit = (
    Circuit(6)
    .add_gate(OpType.CU1, 1.0, [0, 3])
    .add_gate(OpType.CU1, 1.0, [2, 3])
    .add_gate(OpType.CU1, 1.0, [2, 4])
    .H(2)
    .add_gate(OpType.CU1, 1.0, [2, 5])
    .add_gate(OpType.CU1, 1.0, [2, 4])
    .add_gate(OpType.CU1, 1.0, [0, 3])
    .add_gate(OpType.CU1, 1.0, [0, 4])
    .H(3)
    .add_gate(OpType.CU1, 1.0, [0, 5])
    .H(0).Z(0).H(0)
    .add_gate(OpType.CU1, 1.0, [0, 3])
)
# Transform.RebaseToQuil().apply(circuit)
network = NISQNetwork([[0,1]], {0:[0,1,2], 1:[3,4,5]})
render_circuit_jupyter(circuit)

In [ ]:
DQCPass().apply(circuit)
dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)
render_circuit_jupyter(circuit)

In [ ]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [ ]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

In [ ]:
orig_qubits = circuit.qubits
new_qubits = bp_circuit.packed_circuit.qubits
qubit_mapping = {
    orig_qubits[0] : new_qubits[5],
    orig_qubits[1] : new_qubits[6],
    orig_qubits[2] : new_qubits[0],
    orig_qubits[3] : new_qubits[7],
    orig_qubits[4] : new_qubits[1],
    orig_qubits[5] : new_qubits[2],
}
if check_equivalence(circuit, bp_circuit.packed_circuit, qubit_mapping):
    print("The circuit equality has been verified.")
else:
    raise Exception("Circuit equality failed!")